In [1]:
import numpy as np
from PIL import Image as IPIL
from numpy import asarray
import random as pyrand
import asyncio
#from IPython.display import clear_output
import time
import string
import ipywidgets as widgets

import pygtrie

from IPython.display import HTML, display

In [2]:
Nlayers=6
laymask = 0
laymod = 1
layfoundword = 2 
layforgenword = 3
layseed = 4
layseedlen = 5

# mint
# mint[0][ir][ic] : ISinmask       0: no, 1: yes
# mint[1][ir][ic] : ISmod          0: unchanged, 1: changed
# mint[2][ir][ic] : ISfoundword    0:no, 1: yes

# mint[3][ir][ic] : ISforgenword   0:no, 1-9: index in forgen word
# mint[4][ir][ic] : ISseed         0: no, 1: pot seed, 2: active seed, 3: ded seed
# mint[5][ir][ic] : ISseed         0: no, 1: pot seed, 2: active seed, 3: ded seed


#Nbigrows, Nbigcols, Npane, Nalch, IS_showmod, IS_showfoundword 

#class Glob:
#    ialgo = 0
#    ishape = 0
#    ipoem = 0
#    ialgobeg = 0
#    ialgoend = 3
#    Nbigrows = 0
#    Nbigcols = 0

class Inst: 
    def __init__(self):
        self.imgname = ''
        self.poemtit = ''
        self.poemwords = ''
        Nbigrows = 1
        Nbigcols = 1
        self.Npane = 1
        self.Nalch = 5
        self.IS_altrowoffset = False
        self.IS_noblankrow = False
        self.IS_showmod = True
        self.IS_showfoundword = True
        
class Numb: 
    def __init__(self):
        self.chunkrow = 0
        self.chunkcol = 0
        self.Nrowtile = 0
        self.Ncoltile = 0
        self.Nrowtot = 0
        self.Ncoltot = 0
        self.Nrowbuff = 0
        self.Ncolbuff = 0
        self.full_text = []

class State:
    def __init__(self):
        self.fulltext = ''

In [3]:
#def load_words():
#    with open('words_alpha.txt') as word_file:
#        valid_words = set(word_file.read().split())
#    return valid_words

from varname import argname
#import varname

with open('google-10000-english-usa.txt') as word_file:
        valid_words = set(word_file.read().split())

def fcheckword(this_word):
    return (this_word in valid_words)

trie = pygtrie.CharTrie()
for word in valid_words:
    trie[word] = True

#def PR(**args):
def PR(*args,end='\n',flush=False):    
    sout = ''
    i=0
    for arg in args:
        li_vname = argname('*args')
        sout += li_vname[i] + ': ' + str(arg) + '    '
        i+=1
    print(sout,end=end)

class color:
    Bold       = "\033[1m"
    Dim        = "\033[2m"
    Underlined = "\033[4m"
    Blink      = "\033[5m"
    Reverse    = "\033[7m"
    Hidden     = "\033[8m"
    ResetAll   = "\033[0m"

    Default      = "\033[39m"
    Black        = "\033[30m"
    Red          = "\033[31m"
    Green        = "\033[32m"
    Yellow       = "\033[33m"
    Blue         = "\033[34m"
    Magenta      = "\033[35m"
    Cyan         = "\033[36m"
    LightGray    = "\033[37m"
    DarkGray     = "\033[90m"
    LightRed     = "\033[91m"
    LightGreen   = "\033[92m"
    LightYellow  = "\033[93m"
    LightBlue    = "\033[94m"
    LightMagenta = "\033[95m"
    LightCyan    = "\033[96m"
    White        = "\033[97m"

#print('')
#stest = 'ripe'
#print(trie.has_key(stest))
#print(trie.has_node(stest))
#print(trie.has_subtrie(stest))
#print(trie.keys(prefix=stest))

In [4]:
li_imgname = []
li_imgname.append('triangle.bmp')
li_imgname.append('circle_1.bmp')
li_imgname.append('diamonds-tile01.bmp')
li_imgname.append('one-diamond-tile01.bmp')

li_imgname.append('rectangles-tile01.bmp')
li_imgname.append('star01.bmp')
li_imgname.append('L-corner-tile01.bmp')
li_imgname.append('tile-star-tess01.bmp')
#imgname = li_imgname[ishape]
#li_imgname.append('L-square-tile01.bmp')

li_poem=[]
spoem = 'reviver,waking_is_possible__,you_can_hear_the_space_in_words__'
li_poem.append(spoem)

spoem = 'you and now,a_regress_of_spheres_progressing__,hands_clasped__,bridging_the_voids__,between__,blood_travels_forward__,hope_travels_back__'
li_poem.append(spoem)

spoem  = 'level, snow_fields__,hide_and_reveal__,surface-depth__,always_already_there__'
li_poem.append(spoem)

spoem='eve, listen_for__,dreams_of_plants__,the_night_of_winter_solstice__'
li_poem.append(spoem)

spoem='tenet, at_night-time__,answers_are_simply_given__,they_taper_to_thin_whispers__'
li_poem.append(spoem)

In [5]:
BG=open('bigram_A.txt','r')

nbg=np.zeros((26,26))

#            0 1 2 3 4 5 6 7 8 9 1 1 2 3 4 5 6 7 8 9 2 1 2 3 4 5 
salphabet = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'
msalphabet = salphabet.split()

iline=0
for this_line in BG:
    this_words = this_line.split()
    for icol in range(0,26):
        thisrow=ord( this_words[icol*2][0].lower() )  - 97
        thiscol=ord( this_words[icol*2][1].lower() )  - 97
        nbg[thisrow,thiscol] = this_words[icol*2+1]
    iline += 1

if 1 == 0:
    print('{:6}'.format(' ') + ' ',end='')
    for irow in range(0,26):
        print('{:6}'.format(msalphabet[irow]) + ' ',end='')
    print('')
        
    for irow in range(0,26):
        print('{:6}'.format(msalphabet[irow]) + ' ',end='')
        for icol in range(0,26):
            print('{:5.4f}'.format(nbg[irow,icol]) + ' ',end='')
        print('')
    
    this_comb = np.zeros(26)
    this_psum = np.zeros(26)


In [6]:
def fdoimageproc(img_name):
    img = IPIL.open(img_name) # Load image
    
    global imgarray 
    imgarray = asarray(img)
    
    Nrowimg=int(imgarray.shape[0])
    Ncolimg=int(imgarray.shape[1])

    return Nrowimg, Ncolimg, imgarray

In [7]:
def fdomask(tNumb):     
    mtemp = np.zeros((tNumb.Nrowtot+1,tNumb.Ncoltot+1),dtype=int)
    
    irowmask = 0
    icolmask = 0
    
    for irow in range(0,1000):
        if irow % tNumb.chunkrow == 0:
            icolmask = 0
            for icol in range(0,1000):            
                if icol%tNumb.chunkcol==0:
                    if icolmask < tNumb.Ncoltot:                           #why is this necessary?
                        if imgarray[irow,icol,0] > 128:
                            mask[irowmask,icolmask]=0
                        else:
                            mask[irowmask,icolmask]=1  
                    icolmask += 1
            irowmask+=1            
    
    for ibigrow in range(0,thisI.Nbigrows):
        rowoffset = tNumb.Nrowtile*ibigrow
        for irow in range(0,tNumb.Nrowtile):
            iactrow =  rowoffset + irow
            for ibigcol in range(0,thisI.Nbigcols):  
                coloffset = tNumb.Ncoltile*ibigcol
                for icol in range(0,tNumb.Ncoltile):
                    iactcol = coloffset + icol
                    mtemp[iactrow,iactcol] = mask[irow,icol]

    
    return mask, mtemp

In [8]:
def fdonumb(icase):

    locNumb = Numb()
    
    global thisI
    global Nrowtile
    global Ncoltile
    
    thisI = fdochoosecontent(icase) ##########

    locNumb.Nrowtile = int( float(Nrowscreen)/(float(thisI.Nbigrows)))
    locNumb.Ncoltile = int( locNumb.Nrowtile*asprat )     

    Nrowimg, Ncolimg, imgarray = fdoimageproc(thisI.imgname) ##########

    locNumb.chunkrow = int( Nrowimg/float(locNumb.Nrowtile) )
    locNumb.chunkcol = int( Ncolimg/float(locNumb.Ncoltile) )               
    locNumb.Nrowtot = int( float(locNumb.Nrowtile)*thisI.Nbigrows )
    locNumb.Ncoltot = int( float(locNumb.Ncoltile)*thisI.Nbigcols)   
    locNumb.Ncolbuff = int((Ncolscreen - locNumb.Ncoltot*thisI.Npane)/2.0)
    locNumb.Nrowbuff = int((Nrowscreen - locNumb.Nrowtot)/2.0)

    return locNumb

In [9]:
def fmaketext(icase, tNumb):       
    curr_shape=0

    words=thisI.poemwords

    Ntiles = thisI.Nbigrows*thisI.Nbigcols
    Nwords = len(words)
    i_inword = np.zeros(Ntiles,dtype=int)      #current index in word per tile
    
    Nblankrows=0
    iword = 0
    if icase == 1:
        blankline=''
        for iblank in range(0,Ncolscreen):
            blankline+='.'
        for irow in range(0,2):
            li_fulltext.append(blankline)
        for ipoemline in range(1,Nwords):
            words[0] += words[ipoemline]
    for kk in range(0,thisI.Nbigrows):          
        for i in range(0,tNumb.Nrowtile):
            this_line=''
            IS_blankrow = True
            for k in range(0,thisI.Nbigcols):           
                itile = kk*thisI.Nbigcols+k
                itiletxt = itile
                while itiletxt >= len(words):                                         #catch need to start from poem beginning again b/c Ntile > Npoemline
                    itiletxt=itiletxt-len(words)
                for j in range(0,tNumb.Ncoltile):
                    if mask[i,j] == 0:
                        this_line = this_line + fill_int
                    else:
                        IS_blankrow = False
                        this_line += words[itiletxt][i_inword[itile]]
                        i_inword[itile] += 1
                        if i_inword[itile] >= ( len(words[itiletxt]) ):
                            i_inword[itile]=0
            if IS_blankrow:
                Nblankrows+=1
                thisblankrow = this_line
            if kk == 1 and thisI.IS_altrowoffset:
                li_this_line = list(this_line)
                loctemp = []
                for iitem in range(0,len(li_this_line)):
                    loctemp.append(li_this_line[iitem])
                colshift = int(tNumb.Ncoltile/2.0)
                for icol in range(0,thisI.Nbigcols*tNumb.Ncoltile-colshift):
                    li_this_line[icol] = loctemp[icol+colshift]
                for icol in range(0,colshift):
                    li_this_line[icol+thisI.Nbigcols*tNumb.Ncoltile-colshift] = loctemp[icol]                    
                this_line = ''.join(li_this_line)

            if thisI.IS_noblankrow:
                if not IS_blankrow: 
                    li_fulltext.append(this_line)
            else:
                li_fulltext.append(this_line)
    for irow in range(0,Nblankrows):
        li_fulltext.append(thisblankrow)
    if icase == 1:
        for irow in range(0,3):
            li_fulltext.append(blankline)

    
    return li_fulltext

In [10]:
def ffindnewwords(IS_first,plist_startwords, mIS_foundword, tNumb):

    list_newwords = []
    
    if IS_first:
        plist_startwords = []
    
    for irow in range(0,tNumb.Nrowtot-1):
        Nlettinword=np.zeros(tNumb.Ncoltot,dtype='int')
        for icol in range(1,tNumb.Ncoltot-1):
            if mIS_foundword[irow,icol] == 0 and mIS_foundword[irow,icol-1] == 0:
                this_lettinword = 0
                this_str=''
                this_word=''
                if mfulltext[irow,icol].isalpha():
                               
                    this_str = mfulltext[irow,icol]                     #start char is alpha, so set this_str and Nlettersinword
                    this_lettinword = 1 
                    IS_prefix = True
                    IS_word = False
                    while(IS_prefix):                                #loop so find longest word starting with this char
                        prev_str = this_str                         #previous str before add more chars 
                        IS_prefix = False
    
                        if icol+this_lettinword < tNumb.Ncoltot:
                            nextchar = mfulltext[irow,icol+this_lettinword]
                            if nextchar.isalpha():
                                this_str += nextchar
        
                                if(trie.has_subtrie(this_str)):            #If is prefix, keep going to find longest
                                    this_lettinword += 1
                                    IS_prefix=True
        
                                if trie.has_key(this_str):                 #Check all strings if words (ie, even if not longest)
                                    IS_word = True
                                    if IS_first:
                                        if this_str not in plist_startwords:
                                            plist_startwords.append(this_str)
    
                    if not IS_first:
                        if len(prev_str) > 2 and trie.has_key(prev_str):          #loop terminated so either longest word or prefix that is not a word
                            if prev_str not in plist_startwords and prev_str not in list_newwords:
                                #list_newwords.append(prev_str)
                                #Nlettinword[icol] = this_lettinword
                                for ilett in range(0,this_lettinword):
                                        mIS_foundword[irow,icol+ilett] = 1
    #print('')
    return plist_startwords

In [11]:
def fswap(tNumb):
    row1=int(rnGen.integers(low=0, high=tNumb.Nrowtot))
    col1=int(rnGen.integers(low=0, high=tNumb.Ncoltot))
   
    row2=int(rnGen.integers(low=0, high=tNumb.Nrowtot))
    col2=int(rnGen.integers(low=0, high=tNumb.Ncoltot)) 
    temp = mfulltext[row1,col1]
    if IS_swapon:
        mfulltext[row1,col1]=mfulltext[row2,col2]
        mfulltext[row2,col2]=temp

In [12]:
def falch(tNumb):
    
    testrow=int(rnGen.integers(low=1, high=tNumb.Nrowtot-1))
    testcol=int(rnGen.integers( low=1, high=tNumb.Ncoltot-1 ) )   

    this_char = mfulltext[testrow,testcol]
    left_char = mfulltext[testrow,testcol-1]
    righ_char = mfulltext[testrow,testcol+1]
    
    this_alphind = ord(this_char) - 97
    left_alphind = ord(left_char) - 97
    righ_alphind = ord(righ_char) - 97

    loc_Nmod = 0
    this_psum = np.zeros(26)
    if left_alphind >=0 and left_alphind <26:
        if righ_alphind >=0 and righ_alphind <26:

            this_psum[0] = 0.0
            for ilett in range(1,26):
                pforw = nbg[left_alphind,ilett]
                pback = nbg[ilett,righ_alphind]
                this_psum[ilett] = this_psum[ilett-1] + nbg[left_alphind,ilett]*nbg[ilett,righ_alphind]
                
            psumtot = this_psum[26-1]

            rand = rnGen.random()*psumtot

            for ilett in range(0,25):
                if rand > this_psum[ilett] and rand <=  this_psum[ilett+1]:
                    new_char = chr(ilett+97+1)
                    mfulltext[testrow,testcol] = new_char
                    loc_Nmod += 1
                    mint[laymod,testrow,testcol] = 1
    return mint[laymod],mfulltext,loc_Nmod

In [13]:
def fgrowforward(itime,tNumb):

    pnewseed=0.1

    irowpick = -1
    icolpick = -1
    
    IS_new_seed = False     
    rand = rnGen.random()
    if rand < pnewseed:
        IS_new_seed = True

    Npotseed = 0
    if IS_new_seed:        
        li_possseed=[]                                                    #Find pot seeds      
        for irow in range(0,tNumb.Nrowtot):
            for icol in range(1,tNumb.Ncoltot-1):
                cond1 = ( mfulltext[irow,icol-1] =='_' and mfulltext[irow,icol].isalpha() )
                #cond2 = (mfulltext[irow,icol-1] =='.' and mfulltext[irow,icol].isalpha())
                cond3 = (mint[layseed,irow,icol] < 2)
                if cond1 and cond3:
                        mint[layseed,irow,icol] = 1
                        li_possseed.append((irow,icol))          
                        Npotseed += 1
                
        if Npotseed > 0:
            rand = rnGen.integers(0,Npotseed)             #Turn on pot seed
            irowpick = li_possseed[rand][0]
            icolpick = li_possseed[rand][1]
            mint[laymod,irowpick,icolpick] = 1
            mint[layseed,irowpick,icolpick] = 2
            mint[layseedlen,irowpick,icolpick] = 0

    li_actiseed = []
    Nactiseed = 0                                                           #Find active seeds
    for irow in range(0,tNumb.Nrowtot):
        for icol in range(0,tNumb.Ncoltot):
            if mint[layseed,irow,icol] == 2 :
                li_actiseed.append((irow,icol))
                Nactiseed+=1
    
    if Nactiseed > 0:
        seedrow=0
        seedcol=0
        for iseed in range(0,Nactiseed):                                    #loop over active seeds
                seedrow=li_actiseed[iseed][0]
                seedcol=li_actiseed[iseed][1]
                seedlen = mint[layseedlen,seedrow,seedcol]

                this_str='' 
                idelta = 0
                for idelta in range(0,seedlen):
                    this_str += mfulltext[seedrow,seedcol+idelta]
                this_lastcol = seedcol+idelta
                
                if this_lastcol+1 < tNumb.Ncoltot:
                    IS_cangrow = True                                         #check if forgenword can grow
                    if mint[laymask,seedrow,this_lastcol+1] == 0:             #out of shape area so ded seed
                        IS_cangrow = False
                        mint[layseed,seedrow,seedcol] = 3
                                                                       
                    if mint[layseed,seedrow,this_lastcol+1] > 1:              #ran into another seed (active or ded) so ded
                        IS_cangrow = False
                        mint[layseed,seedrow,seedcol] = 3
    
                    if IS_cangrow:
                        if trie.has_subtrie(this_str):                        #can grow, but is there a longer word or at end of branch?
                            li_prefix = trie.items(this_str)                  #list of possibilities for this prefix
                            if li_prefix[0] == this_str:
                                rand = rnGen.integers(1,len(li_prefix))       #avoid picking shorter word (current this_str)
                            else:
                                rand = rnGen.integers(0,len(li_prefix))       #this_str has only 1 word, so grow to full length bit by bit
                            this_word = li_prefix[rand][0]
                            if len(this_word) > seedlen:                      #longer words so grow
                                mfulltext[seedrow,seedcol+seedlen] = this_word[seedlen]
                                mint[layseedlen,seedrow,seedcol] += 1
                                mint[laymod,seedrow,seedcol+seedlen] = 1

                        else:
                            mint[layseed,seedrow,seedcol] = 3              #no longer words so ded
                            IS_needclear = True
                            icolsub = seedcol+seedlen
                            while IS_needclear and icolsub < tNumb.Ncoltot:
                                checkchar = mfulltext[seedrow,icolsub]
                                if checkchar != '_' and checkchar != '.':
                                    #mfulltext[seedrow,seedcol+seedlen+ineedclear] = '_'
                                    mfulltext[seedrow,icolsub] = '_'
                                else:
                                    IS_needclear = False
                                #ineedclear+=1
                                icolsub+=1

    return mfulltext, mint

In [14]:
def fdochoosecontent(icase):
    LocI=Inst()

    if icase == 0:
        LocI.Nbigrows = 2
        LocI.Nbigcols = 3        
        LocI.Npane=1 
        LocI.Nalch = 10
        LocI.IS_showmod = False
        LocI.IS_showfoundword = True
    if icase == 1:
        LocI.Nbigrows = 1
        LocI.Nbigcols = 1        
        LocI.Npane=2
        LocI.Nalch = 10
        LocI.IS_showmod = False
        LocI.IS_showfoundword = True        
    if icase == 2:
        LocI.Nbigrows = 2
        LocI.Nbigcols = 3        
        LocI.Npane=1
        LocI.Nalch = 5
        LocI.IS_showmod = True
        LocI.IS_showfoundword = False
    

    LocI.imgname = li_imgname[ishape]

    lipoemsplit = li_poem[ipoem].split(',')
    LocI.poemtit = lipoemsplit[0]
    LocI.poemwords = []
    for iline in range(1,len(lipoemsplit)):
        LocI.poemwords.append(lipoemsplit[iline])

    LocI.IS_altrowoffset = False
    LocI.IS_noblankrow = False
    if ishape == 2:
        LocI.IS_noblankrow = True    
    if ishape == 5:
        LocI.IS_altrowoffset = True
        LocI.IS_noblankrow = True

    return LocI

In [15]:
def fprintoutput(itime,icase,tNumb):
            sintervaltime = str(int(itime/10.0)*10)
            currscreen = ''   
    
            for irow in range(0,tNumb.Nrowtot):
                if irow<41:
                    this_line = ''
                    for icol in range(0,tNumb.Ncolbuff):
                        this_line += '.'
                    for ipane in range(0,thisI.Npane):
                        for icol in range(0,tNumb.Ncoltot):  
                            IS_default = True
                            if mint[layfoundword,irow,icol]==1:
                                if thisI.IS_showfoundword:
                                    #this_line+=color.Blue+color.Bold+mfulltext[irow,icol]+color.ResetAll
                                    this_line+='<span style="color: blue; font-weight: bold;">'+mfulltext[irow,icol]+'</span>'
                                    IS_default = False
                                                                   
                            elif mint[laymod,irow,icol]>0:
                                if thisI.IS_showmod:
                                    #this_line+=color.LightMagenta+color.Bold+mfulltext[irow,icol]+color.ResetAll
                                    this_line+='<span style="color: green; font-weight: bold;">'+mfulltext[irow,icol]+'</span>'
                                    IS_default = False
         
                            if IS_default:
                                if ipane == 1:                            
                                    this_line += '.'
                                else:
                                    this_line += mfulltext[irow,icol]
                    ilastcol=icol 
                    for icol in range(0,tNumb.Ncolbuff):
                        this_line += '.'
                    if irow==40:
                        currscreen+=this_line
                    else:
                        currscreen+=this_line+'<BR>'
                    ilastrow = irow
                    #with out:
                    #    print(this_line)
            #with wout:               
                #wout.clear_output(wait=True)
            #print('live generation algorithm: ' + color.Yellow+color.Bold+mscase[icase]+color.ResetAll 
            #      + '              random seed: ' + color.Yellow+color.Bold + rand_word +color.ResetAll  
            #      + '              poem: ' + color.Yellow+color.Bold + thisI.poemtit+color.ResetAll #)
            #        + '                                                            ' 
            #      + color.Yellow+color.Bold + sintervaltime+color.ResetAll )                
            #print(currscreen,flush=True)
            #wout.value = f"<pref>{spre} {currscreen} {spos}</pre>"
            #wout.value = f"<pref> {currscreen} </pre>"
            #await asyncio.sleep(0.15)
            return currscreen
            #display(currscreen)
            #display(currscreen)
            #for irow in range(ilastrow,Nrowscreen):
            #    this_line = ''
            #    for icol in range(0,Ncolscreen):
            #        this_line += '.'                
            #    print(this_line)

In [16]:
async def fdorun():
    
    ialgobeg=0
    ialgoend=3

    iiter = 0

    global tmax
    
    tNumb = Numb()

    global mint
    global mask
    global li_fulltext 
    global mfulltext

    global ialgo
    global ishape
    global ipoem

    spre = '<div style="font-family: \'Courier\', sans-serif; font-size: 13px; color: black; font-weight: normal; line-height: 1.2;">'
    spos = '</div>'     

    sbeg = '<span style="color: blue; font-weight: bold;">'
    send = '</span>'
    spac = '&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'

    sbegdor = '<span style="color: chocolate; font-weight: bold;">'

    
    while True:
           
        if IS_htmlonly:
            aspratrow, aspratcol, nfontsize, icase, ishape, ipoem, shtmlfilename, tmax, IS_keepgoing = fhtmlsetup(ipair)
            ipair += 1
        
        if radiomode.value == 'random auto':
            ialgobeg=0
            ialgoend=3
            
        else:
            ialgobeg=dropalgo.value
            ialgoend=ialgobeg+1  
            
            
        for icase in range(ialgobeg,ialgoend):
            if radiomode.value == 'random auto':
                ishape = rnGen.integers(1,7)
                ipoem = rnGen.integers(0,4)
                tmax = islidtmax.value
            else:                
                ishape=dropshape.value
                ipoem=droppoem.value
                tmax =  100000
              
            Nmod = 0
            loc_Nmod = 0

            tNumb = fdonumb(icase) ##############################

            wban.value = f'{spre} algorithm: {sbegdor}{mscase[icase]}{send} {spac} random seed: {sbegdor}{rand_word}{send} {spac} poem: {sbegdor}{thisI.poemtit}{send} {spos}' 
            #print('live generation algorithm: ' + color.Yellow+color.Bold+mscase[icase]+color.ResetAll 
            #      + '              random seed: ' + color.Yellow+color.Bold + rand_word +color.ResetAll  
            #      + '              poem: ' + color.Yellow+color.Bold + thisI.poemtit+color.ResetAll #)
            #        + '                                                            ' 
            #      + color.Yellow+color.Bold + sintervaltime+color.ResetAll )  
            

            mint = np.zeros((Nlayers,tNumb.Nrowtot+1,tNumb.Ncoltot+1),dtype=int)
            mint.fill(0)
            mfulltext = np.zeros((tNumb.Nrowtot+1,tNumb.Ncoltot+1),dtype=str)        
            mask=np.zeros((tNumb.Nrowtot+1,tNumb.Ncoltot+1),dtype=int)
            mask, mint[laymask] = fdomask(tNumb) ##################################   
            li_fulltext=[]
            
            li_fulltext = fmaketext(icase, tNumb) ##########
            
            for irow in range(0,tNumb.Nrowtot):
                for icol in range(0,tNumb.Ncoltot):
                    mfulltext[irow,icol] = li_fulltext[irow][icol]            
            mfulltext_orig = mfulltext       
    
            if IS_htmlonly:
                fdoHTML(shtmlfilename) ##########      
            
            list_startwords = ffindnewwords(True,[],mint[layfoundword],tNumb) ##########
            
            for itime in range(0,tmax):
                if ISgoevent.is_set():
                    for k in range(0,thisI.Nalch):
                        if icase == 0:
                            fswap(tNumb)
                            mint[layfoundword].fill(0)
                            list_startwords = ffindnewwords(False,list_startwords,mint[layfoundword],tNumb) ##########                     
                        if icase == 1:
                            mint[laymod],mfulltext,loc_Nmod = falch(tNumb) ##########
                            mint[layfoundword].fill(0)
                            list_startwords = ffindnewwords(False,list_startwords,mint[layfoundword],tNumb) ##########
                        if icase == 2:
                            mfulltext, mint = fgrowforward(itime,tNumb) ##########
                        Nmod += loc_Nmod
                    #wout.value = f'test {itime}' 
                    currscreen = fprintoutput(itime,icase,tNumb) ##########
                    wout.value = f'{spre} {currscreen} {spos}' 
                    #wout.value = f'{currscreen}' 
                    await asyncio.sleep(0.05)
                else:
                    while not ISgoevent.is_set():
                        #wout.value = f"<pref>dur  {i}</pre>"
                        await asyncio.sleep(0.25)                   
                
            iiter+=1

In [17]:
def fmakewidgets(): 
    global buttstart
    global buttpause
    global radiomode
    global htmlmode
    global islidtmax
    global droppoem
    global dropalgo
    global dropshape


    buttstart = widgets.Button(
        description = 'start'
        ,layout=widgets.Layout(width='100px', height='40px')
    )
    buttstart.on_click(OBCstart)

    buttpause = widgets.Button(
        description = 'pause'
        ,disabled = True
        ,layout=widgets.Layout(width='100px', height='40px')   
    )
    buttpause.on_click(OBCpause)

    htmlmode = widgets.HTML(
        value="select mode: &nbsp;&nbsp;",
        placeholder='',
        description='',
    )
    
    radiomode = widgets.RadioButtons(
        options=['random auto', 'manual select'],
    #    value='pineapple', # Defaults to 'pineapple'
        #layout={'flex-direction': 'row' }, # If the items' names are long
        #description='Mode:',
        disabled=False
        ,layout=widgets.Layout(width='200px', height='40px')
    )
    radiomode.observe(on_radio_button_change, names='value')

    islidtmax = widgets.IntSlider(
        value=300,
        min=100,
        max=1000,
        step=100,
        description='Time per algo/poem/shape combo:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )    

    droppoem = widgets.Dropdown(
        options=[('reviver', 0), ('you and then', 1), ('level', 2), ('eve', 3), ('tenet', 4)],
        value=2,
        description='Poem:',
    )
    
    dropshape = widgets.Dropdown(
        options=[('triangle', 0), ('circle', 1), ('diamonds', 2), ('1 diamond', 3), ('rectangle', 4), ('star', 5), ('L corner', 6)],
        value=2,
        description='Shape:',
    )
    
    dropalgo = widgets.Dropdown(
        options=[('swap', 0), ('alch', 1), ('growforward', 2)],
        value=2,
        description='Shape:',
    )

In [18]:


#wd = widgets.interact(slider, x=widgets.BoundedIntText(min=0, max=len(result_cat)-1))

asprat= 28.0/13.0

fill_int='.'
fill_ext='.'
len_ext_fill = 0
nfontsize=2.0

tmax=300

Nrowscreen = 40
Ncolscreen = 180
Ncolscreen = 167

IS_swapon = True
IS_htmlonly = False
IS_manual = False

Nrowtot = 0

wtop = widgets.Output(layout={'border': '1px solid black', 'height': '45px', 'width' : '40%', 'overflow': 'none'})
wtopR = widgets.Output(layout={'border': '1px solid black', 'height': '45px', 'width' : '60%', 'overflow': 'none'})
wban = widgets.HTML(layout={'border': '1px solid black', 'height': '20px'})
wout = widgets.HTML(layout={'border': '1px solid black', 'height': '770px'})
hboxtop = widgets.HBox([wtop, wtopR])
display(hboxtop,wban,wout)

loop = asyncio.get_event_loop()
ISgoevent = asyncio.Event()

thisI=Inst()

Ncase = 3
mscase=['swap','alch','growforward']

#Random seed
rand = np.random.default_rng().integers(0,len(valid_words))
rand_word = list(valid_words)[rand]
rnGenseed = pyrand.seed(rand_word)
rnGen = np.random.default_rng(seed=rnGenseed)

#global ttest 
ttest = None

#######################################
def OBCstart(button_instance):     
    global ttest
    ISgoevent.set()
    if ttest == None or ttest.done():
        buttpause.disabled = False
        ttest = loop.create_task(fdorun())
        buttstart.description = 'stop & reset'
        buttpause.description = 'pause'
    else:
        ttest.cancel()
        buttpause.disabled = True
        buttstart.description = 'start'
        buttpause.description = 'pause'
        
def OBCpause(button_instance):
    global ttest
    #global ISgoevent
    if ISgoevent.is_set():
        ISgoevent.clear()
        buttpause.description = 'unpause'
    else:
        buttpause.description = 'pause'        
        ISgoevent.set()

def on_radio_button_change(change):
    if radiomode.value == 'random auto':
        wtopR.outputs = ()
        hboxtopR = widgets.HBox([islidtmax])
        wtopR.append_display_data(hboxtopR)        
    else:
        wtopR.outputs = ()
        hboxtopR = widgets.HBox([droppoem, dropshape, dropalgo])
        wtopR.append_display_data(hboxtopR)
        #wout.value= f"Radio"       

#def OBCstop(button_instance):     
#    global ttest
#    global ISgoevent
#    ttest.cancel() 
#    buttstart.disabled = False

########################################







fmakewidgets()





#buttstop = widgets.Button(
#    description = 'stop'
#)
#buttstop.on_click(OBCstop)
#######################################




def main_loop():
    hboxtopL = widgets.HBox([buttstart, buttpause, htmlmode, radiomode])
    hboxtopR = widgets.HBox([islidtmax])
    wtop.append_display_data(hboxtopL) 
    wtopR.append_display_data(hboxtopR) 

if __name__ == "__main__":   
    main_loop()
    

HTML(value='', layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px…

HTML(value='', layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px…

In [19]:
style = """
    <style>
       .jupyter-widgets-output-area .output_scroll {
            height: unset !important;
            border-radius: unset !important;
            -webkit-box-shadow: unset !important;
            box-shadow: unset !important;
        }
        .jupyter-widgets-output-area  {
            height: auto !important;
        }
    </style>
    """
horizontal_style = HTML("""
    <style>
        .widget-radio-box {
            flex-direction: row !important;
        }
        .widget-radio-box label {
            margin: 5px !important; /* Adjust as needed */
            width: auto !important; /* Adjust as needed */
        }
    </style>
""")


display(HTML(style))
display(widgets.HTML(style))

HTML(value='\n    <style>\n       .jupyter-widgets-output-area .output_scroll {\n            height: unset !im…